In [78]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import winsound
import datetime as dt

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer, StandardScaler

import tensorflow as tf
print(f" Found and Using {len(tf.config.experimental.list_physical_devices('GPU'))} GPU")

 Found and Using 1 GPU


In [79]:
def scale_data(df):

    scaled_features = StandardScaler().fit_transform(df.values)
    df = pd.DataFrame(scaled_features, index=df.index, columns=df.columns)
 
    return(df)

In [80]:
#df = pd.read_csv('data/tab_df.csv')
df = pd.read_csv('data/IRIS.csv')

print(len(df))
#df = df.drop(['month', 'day', 'hour', 'min'], axis=1)
df.head(10)

150


,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
5,5.4,3.9,1.7,0.4,Iris-setosa
6,4.6,3.4,1.4,0.3,Iris-setosa
7,5.0,3.4,1.5,0.2,Iris-setosa
8,4.4,2.9,1.4,0.2,Iris-setosa
9,4.9,3.1,1.5,0.1,Iris-setosa


In [81]:
df['species'].value_counts()

Iris-setosa        50
Iris-versicolor    50
Iris-virginica     50
Name: species, dtype: int64

In [82]:
new_targets = pd.get_dummies(df['species'])

In [83]:
df = df.join(new_targets)
del df['species']
df.head()

,sepal_length,sepal_width,petal_length,petal_width,Iris-setosa,Iris-versicolor,Iris-virginica
0,5.1,3.5,1.4,0.2,1,0,0
1,4.9,3.0,1.4,0.2,1,0,0
2,4.7,3.2,1.3,0.2,1,0,0
3,4.6,3.1,1.5,0.2,1,0,0
4,5.0,3.6,1.4,0.2,1,0,0


mapping = {'Iris-setosa': 0, 'Iris-versicolor': 1, 'Iris-virginica': 2}
df = df.replace({'species': mapping})

#mapping = {'decrease': 0, 'same': 1, 'increase': 2}
#df = df.replace({'target': mapping})
df.head()

In [75]:
l = [1,2,3,4]

if 4 not in l:
    print('worked')

target_col = 'species'

train_x, valid_x, train_y, valid_y = train_test_split(df.loc[:, df.columns != target_col], df[target_col],train_size = 0.8,random_state = 1)

In [84]:
input_cols = ['sepal_length','sepal_width','petal_length','petal_width',]
target_cols = ['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']


train_x, valid_x, train_y, valid_y = train_test_split(df.loc[:, input_cols], df[target_cols],train_size = 0.8,random_state = 1)

In [55]:
#train_x = scale_data(train_x)
#valid_x = scale_data(valid_x)
#train_x.head()

In [87]:
Layer_nodes = 4

model = tf.keras.Sequential([
  tf.keras.layers.Dense(Layer_nodes, activation='relu', input_shape=[len(train_x.keys())]),
  tf.keras.layers.Dense(Layer_nodes, activation='relu'),              
#  tf.keras.layers.Dense(Layer_nodes, activation='relu'),      
#  tf.keras.layers.Dense(Layer_nodes, activation='relu'),      
#  tf.keras.layers.Dense(Layer_nodes, activation='relu'),      
  tf.keras.layers.Dense(3, activation='softmax')
])

In [92]:
model = tf.keras.Sequential([
  tf.keras.layers.Dense(Layer_nodes, activation='relu', input_shape=[len(train_x.keys())]),
  tf.keras.layers.Dense(3, activation='softmax')
])

In [93]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',    
    metrics=['accuracy'])

In [94]:
epoch_val = 3

In [95]:
model.fit(train_x, train_y, epochs=epoch_val)

Epoch 1/3
4/4 [==============================] - 0s 6ms/step - loss: 3.4555 - accuracy: 0.3083
Epoch 2/3
4/4 [==============================] - 0s 6ms/step - loss: 3.3565 - accuracy: 0.3083
Epoch 3/3
4/4 [==============================] - 0s 6ms/step - loss: 3.2596 - accuracy: 0.3083


In [18]:
test_loss, test_accuracy = model.evaluate(valid_x, valid_y)
print(f'Test Loss {test_loss}')
print(f'Test Accuracy {test_accuracy}')

1/1 [==============================] - 0s 0s/step - loss: 0.0249 - accuracy: 0.4333
Test Loss 0.024903973564505577
Test Accuracy 0.4333333373069763


In [96]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [97]:
clf = RandomForestClassifier()
clf.fit(train_x, train_y)

RandomForestClassifier()

In [104]:
print(classification_report(valid_y, clf.predict(valid_x),target_names=target_cols))

                 precision    recall  f1-score   support

    Iris-setosa       1.00      1.00      1.00        11
Iris-versicolor       1.00      0.92      0.96        13
 Iris-virginica       0.86      1.00      0.92         6

      micro avg       0.97      0.97      0.97        30
      macro avg       0.95      0.97      0.96        30
   weighted avg       0.97      0.97      0.97        30
    samples avg       0.97      0.97      0.97        30

